In [3]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
import numpy
import nltk

from scipy import spatial

from scipy import stats
from scipy.stats import pearsonr,spearmanr

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.


In [2]:
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import *
import numpy as np


#load pre-trained vectors
model_b = Word2Vec.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary=True) #address of the vectors
model_b.init_sims(replace=True)


In [7]:
import nltk


#this function takes the address of the data and processes it
def make_the_XandY(address, max_length=25):
    
    paracorpus = open(address).read() #reads a corpus of paraphrases 
    
    paragroups = paracorpus.split("\n\n")  #splits it in groups of paraphrases

    parasent = [group.split("\n") for group in paragroups] #splits the groups in sentences

    XandY_train = [[el.split("#") for el in group] for group in parasent]

    XandY_train = [[el for el in group if len(el[0])>1] for group in XandY_train]
    
    XandY_train = [el for el in XandY_train if len(el)>=5] #filters out spurious elements

    print("len x and y" ,len(XandY_train))
    
    return XandY_train




def make_the_binary(XandY_train, max_length=25):
    
    Xa_text, Xb_text, myYs = [], [], []
    
    for i in range(0,len(XandY_train[:])):
        
        group = XandY_train[i]

        for el in group:
            
            prv = []
            
            if len(el)>1: 
                
                Xa_text.append(group[0][0])
                
                Xb_text.append(el[0])
                
                if el[-1].strip().isdigit(): myYs.append(int(el[-1].strip()))
                    
                else: print("WARNING: there is no Y\n",el[0])

    print("I : ", i, "xandy: ", len(XandY_train))       

    Xa0 = np.array([[model_b[nltk.wordpunct_tokenize(t)[i].lower()] for i in range(len(t.split())) if nltk.wordpunct_tokenize(t)[i].lower() in model_b.vocab] for t in Xa_text])
    
    Xb0 = np.array([[model_b[nltk.wordpunct_tokenize(t)[i].lower()] for i in range(len(t.split())) if nltk.wordpunct_tokenize(t)[i].lower() in model_b.vocab] for t in Xb_text])

    #print(len(Xa0),len(Xb0))

    Xa = sequence.pad_sequences(Xa0, maxlen=max_length, dtype='float32')
    
    Xb = sequence.pad_sequences(Xb0, maxlen=max_length, dtype='float32')
    
    Ys =[]
    
    for el in myYs: 
        
        if el <3 : Ys.append(0.)
            
        else: Ys.append(1.)
        
    print("len of labels: ",len(Ys))

    #print(myYs[:10])
    
    return Xa,Xb,Ys

In [9]:
#  training data

XandYstuff = make_the_XandY("metaphor_paraphrase_corpus")  #address of the corpus

#shuffle
#random.shuffle(XandYstuff)

print("len: ",len(XandYstuff))
x_a, x_b, y_ = make_the_binary(XandYstuff)

s = int(.92*len(x_a)) #validation split

print("training size:",s)

print("total corpus size", len(x_a))

x_train_a, x_train_b, y_train = x_a[:s], x_b[:s], y_[:s]

x_val_a, x_val_b, y_val = x_a[s:], x_b[s:], y_[s:]



len x and y 187
len:  187
I :  186 xandy:  187
len of labels:  744
training size: 684
total corpus size 744


In [10]:
len(y_train), len(x_train_a), len(y_val), len(x_val_a)

(684, 684, 60, 60)

In [11]:
data_dim = 300

timesteps = max_length = 25

#nb_classes = 2

In [18]:
#Model 
from keras.models import Sequential, Model
from keras.layers import Merge, LSTM, Dense, Flatten, Input, merge
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Convolution1D, AtrousConvolution1D
from keras.layers import Lambda, Activation, Dropout, Embedding, TimeDistributed
from keras.regularizers import l2
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


def mymodel():
        
    np.random.seed(7) 

    encoder = Sequential()

    encoder.add(AtrousConvolution1D(nb_filter=25, filter_length=5,input_shape=(timesteps, data_dim), 
                                      border_mode='same', activation='relu'))
    
    encoder.add(MaxPooling1D(pool_length=2))
    

    encoder.add(LSTM(20, input_shape=(timesteps, data_dim))) 
    
    encoder.add(Dense(15,input_shape=(timesteps, data_dim))) 
    
    encoder.add(Dropout(.5))
    
    encoder.add(Dense(10, init='he_normal', W_regularizer=l2(0.2))) 

    a = Input((timesteps, data_dim))
    
    b = Input((timesteps, data_dim))

    encoder_2 = Model(input=[a, b], output=merge([encoder(a), encoder(b)], mode='concat'))
 
    decoder_ = Sequential()
    
    decoder_.add(Dense(1, activation='sigmoid', input_shape=(2*encoder.output_shape[-1],))) 

    decoder = Model(input=[a, b], output=decoder_(encoder_2([a, b])))

    opti = optimizers.Adadelta()
    #opti = 'rmsprop'
    
    decoder.compile(loss='binary_crossentropy', 
                    optimizer=opti,
                    metrics=['accuracy']) 

    decoder.summary()
    return decoder

decoder = mymodel()

/Users/xbizyu/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:741: UserWarning: The `AtrousConvolution1D` layer  has been deprecated. Use instead the `Conv1D` layer with the `dilation_rate` argument.
  warnings.warn('The `AtrousConvolution1D` layer '
/Users/xbizyu/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:745: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(25, 300), filters=25, activation="relu", padding="same", kernel_size=5, dilation_rate=1)`
  return Conv1D(*args, **kwargs)
/Users/xbizyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
/Users/xbizyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 25, 300)      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 25, 300)      0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 20)           41680       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 1)            21          model_5[1][0]                    
Total para

/Users/xbizyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/xbizyu/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/xbizyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
/Users/xbizyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("se...)`


In [ ]:
#Binary task

In [19]:
decoder.fit([x_train_a[:], x_train_b[:]], y_train[:],
            batch_size=64, epochs=130, shuffle=False)

Epoch 1/130
684/684 [==============================] - 4s 6ms/step - loss: 14.9184 - acc: 0.4722
Epoch 2/130
684/684 [==============================] - 1s 1ms/step - loss: 14.5657 - acc: 0.5029
Epoch 3/130
684/684 [==============================] - 1s 1ms/step - loss: 14.2168 - acc: 0.5526
Epoch 4/130
684/684 [==============================] - 1s 1ms/step - loss: 13.8738 - acc: 0.5512
Epoch 5/130
684/684 [==============================] - 1s 2ms/step - loss: 13.5360 - acc: 0.5746
Epoch 6/130
684/684 [==============================] - 2s 2ms/step - loss: 13.2037 - acc: 0.6140
Epoch 7/130
684/684 [==============================] - 1s 2ms/step - loss: 12.8803 - acc: 0.5965
Epoch 8/130
684/684 [==============================] - 1s 2ms/step - loss: 12.5605 - acc: 0.6301
Epoch 9/130
684/684 [==============================] - 1s 2ms/step - loss: 12.2464 - acc: 0.6491
Epoch 10/130
684/684 [==============================] - 1s 2ms/step - loss: 11.9382 - acc: 0.6491
Epoch 11/130
684/684 [=======

In [20]:
decoder.evaluate([x_val_a, x_val_b], y_val)

60/60 [==============================] - 1s 11ms/step


[1.2685597260793051, 0.78333332538604739]

In [ ]:
#Ordering task

In [23]:
max_length = timesteps


def ordering(decoder,XandY):
    c=0
    
    whatwewant, whatwehave = [],[]
    
    baseline = []
    
    Xa_text, Xb_text, myYs = [], [], []
    
    for group in XandY[1:]:
        
        c+=1

        if len(group)<5: pass
        
        t = group[0][0].lower()

        sentence0 = [model_b[t.split()[i]] for i in range(len(t.split())) if t.split()[i] in model_b.vocab]
        
        a = np.array([el for el in sentence0]).sum(axis=0) #for baseline
        
        sentence = np.array([sentence0])
        
        sentence = sequence.pad_sequences(sentence, maxlen=max_length, dtype='float32')

        prv = []
        
        prv_baseline = []
        
        for el in group:
            
            if len(el)>1:
                
                t2 = el[0].lower()
                
                compar0 = [model_b[t2.split()[i]] for i in range(len(t2.split())) if t2.split()[i] in model_b.vocab]

                b = np.array([el for el in compar0]).sum(axis=0) #for baseline
                
                compar = np.array([compar0])
                
                compar = sequence.pad_sequences(compar, maxlen=max_length, dtype='float32')

                prv.append((decoder.predict([sentence,compar])[0],int(el[-1]),t2))
                
                prv_baseline.append(1 - spatial.distance.cosine(a,b)) #for baseline
                
        prv.sort()
        
        want,have = [],[]
        
        for x in prv: 

            want.append(x[1])
            
            have.append(x[0][0])
            
        whatwewant.append(want)
        
        whatwehave.append(have)
        
        baseline.append(prv_baseline)
        
    print(len(myYs))
    
    return whatwewant,whatwehave,baseline

whatwewant,whatwehave,baseline = ordering(decoder, XandYstuff)

0


In [27]:
#pearson and spearman mean correlations
def pea_and_spea(whatwewant,whatwehave,baseline):
    
    allpea, allspe,basisP, basisS= [],[],[],[]
    
    for i in range(len(whatwewant)):
        
        pear= pearsonr(whatwewant[i],whatwehave[i])[0]
        
        spe = spearmanr(whatwewant[i],whatwehave[i])[0]
        
        base_pear = pearsonr(whatwewant[i],baseline[i])[0]
        
        base_spe = spearmanr(whatwewant[i],baseline[i])[0]
         
        # the >-1000 precaution is to avoid NaNs and similar occurrences
        
        if pear>-1000: allpea.append(pear)
            
        if spe>-1000: allspe.append(spe)
            
        if base_pear>-1000: basisP.append(base_pear)
            
        if base_spe>-1000: basisS.append(base_spe)
            
    return np.mean(allpea), np.mean(allspe), np.mean(basisP),np.mean(basisS)

correlations = pea_and_spea(whatwewant,whatwehave, baseline)

/Users/xbizyu/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:2998: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
/Users/xbizyu/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:2999: RuntimeWarning: Mean of empty slice.
  my = y.mean()


In [28]:
print("mean pearson correlation: ", correlations[0])
print("mean spearman correlation: ", correlations[1])
print("mean pearson correlation for baseline (cosine similarity of sentences): ", correlations[2])
print("mean spearman correlation for baseline (cosine similarity of sentences): ", correlations[2])

mean pearson correlation:  0.800945046323
mean spearman correlation:  0.72821281224
mean pearson correlation for baseline (cosine similarity of sentences):  -0.0272379148804
mean spearman correlation for baseline (cosine similarity of sentences):  -0.0272379148804
